# Construct Data Set


In [1]:
import pandas as pd
import numpy as np
import os
import time

### Merge Dataset pulled from LexisNexis containing sentiment scores with Event Ratings Data

In [2]:
sentiment_path = "/Users/laurieottehenning/Documents/Georgetown Data Science /Capstone"
os.chdir(sentiment_path)
df1 = pd.read_csv('Sentiment Data.csv', parse_dates=['date'])

event_path = "/Users/laurieottehenning/Documents/Georgetown Data Science /Capstone/Events Ratings"
os.chdir(event_path)
df2 = pd.read_csv('Events_FinalRatingsList.csv', parse_dates=['Date'])

In [3]:
df1 = df1.rename(columns={'date': 'Date'})
df1['Date'].head(3)

0   2010-01-03
1   2010-01-04
2   2010-01-08
Name: Date, dtype: datetime64[ns]

In [4]:
df2['Date'].head(3)

0   2011-01-31
1   2011-03-11
2   2011-03-12
Name: Date, dtype: datetime64[ns]

### Merge the Sentiment data onto the Event data

In [5]:
# Look for duplicate values in event data
df2['dup'] = df2.sort_values(by=['Date']).duplicated(subset=['Date', 'EventID'])
df2.iloc[10:15]

,EventID,Date,EventText,FilterWords,FinalRating,dup
10,1053,2011-04-25,"The events of 25 April, especially those in Da...","all, nation",-1.0,False
11,1055,2011-04-27,According to a report of refugees fled into Le...,"accord, leban",-1.0,False
12,2004,2011-05-06,"On 6 May, after Friday noon prayers, demonstra...","accord, kurd",-1.0,False
13,2005,2011-05-06,"Following this ""Day of Defiance"", Amnesty Inte...","defianc, nation",-1.0,False
14,2009,2011-05-09,The European Union institutes arms embargo and...,"embargo, capitalletters",2.0,False


In [6]:
# Look at counts for events
df2.count()

EventID        548
Date           548
EventText      548
FilterWords    548
FinalRating    548
dup            548
dtype: int64

In [7]:
# Look at unique events by date
df2['Date'].value_counts().count()

442

In [8]:
# Drop duplicate events. We should only have one event per day. 
event = df2.drop_duplicates(subset=['Date'], keep='first')
event.count()

EventID        442
Date           442
EventText      442
FilterWords    442
FinalRating    442
dup            442
dtype: int64

In [9]:
# Drop duplicates column
event = event.drop('dup', 1)


In [27]:
final = pd.merge(event, df1, on='Date', how='outer', indicator=True)
final = final.drop('Unnamed: 0', 1)
final.sort_values(by='Date').iloc[510:520]

,EventID,Date,EventText,FilterWords,FinalRating,publication,title,length,publicationtype,text,year,month,day,PositiveCount,NegativeCount,tone,_merge
2897,NaN,2011-05-15,NaN,NaN,NaN,BBC,Programme summary of Afghan Noor TV news in Da...,701.0,Transcript,A News headline B Home news 1 0100 Russian for...,2011.0,5.0,15.0,28.0,46.0,-0.025678,right_only
2899,NaN,2011-05-15,NaN,NaN,NaN,Korea Times,The 'Obama Doctrine' is a fuzzy document,590.0,Newspaper,Barack Obama is a president in search of a doc...,2011.0,5.0,15.0,33.0,40.0,-0.011864,right_only
54,2017.0,2011-05-16,The Syrian army's siege across the country con...,"all, leban",2.0,BBC,"Gulf council's membership offer to Jordan ""sur...",572.0,Transcript,Text of report by Londonbased newspaper AlHaya...,2011.0,5.0,16.0,26.0,16.0,0.017483,both
53,2017.0,2011-05-16,The Syrian army's siege across the country con...,"all, leban",2.0,Usa Today,"Syria, Libya merit different U.S. policies; Da...",864.0,Newspaper,When Libyas dictator Moammar Gadhafi attacked ...,2011.0,5.0,16.0,34.0,52.0,-0.020833,both
56,2017.0,2011-05-16,The Syrian army's siege across the country con...,"all, leban",2.0,BBC,"Column denies Turkish ""regional leadership"" af...",821.0,Transcript,Text of report in English by Turkish newspaper...,2011.0,5.0,16.0,42.0,52.0,-0.012180,both
55,2017.0,2011-05-16,The Syrian army's siege across the country con...,"all, leban",2.0,The Washington Post,The U.S.-Saudi split,1003.0,Newspaper,A tectonic shift has occurred in the USSaudi r...,2011.0,5.0,16.0,51.0,53.0,-0.001994,both
2902,NaN,2011-05-17,NaN,NaN,NaN,The New York Times,Questions Raised About a Code of Silence,1289.0,Newspaper,PARIS The ritual follows a clear script a sca...,2011.0,5.0,17.0,60.0,81.0,-0.016292,right_only
2901,NaN,2011-05-17,NaN,NaN,NaN,The Irish Times,Arab League appoints El-Arabi,473.0,Newspaper,EGYPT S FORMER foreign minister Nabil elArabi ...,2011.0,5.0,17.0,23.0,31.0,-0.016913,right_only
2907,NaN,2011-05-18,NaN,NaN,NaN,BBC,"Commentator says pro-Palestinian march ""test"" ...",604.0,Transcript,Text of report by Saudiowned leading panArab d...,2011.0,5.0,18.0,26.0,49.0,-0.038079,right_only
2906,NaN,2011-05-18,NaN,NaN,NaN,The Guardian,Mission impossible: Barack Obama's Middle East...,968.0,Newspaper,Simon Tisdall Obama will have to call on all h...,2011.0,5.0,18.0,48.0,61.0,-0.013430,right_only


In [28]:
# Number of unique events. Spot check.
final.groupby(['EventID', 'Date']).size().count()

442

In [29]:
# Export to csv
merged_path = "/Users/laurieottehenning/Documents/Georgetown Data Science /Capstone/Merged"
os.chdir(merged_path)
final.to_csv('Merged Events_Sentiment.csv')